In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('exercise_dataset.csv')
df

,ID,Exercise,Calories Burn,Dream Weight,Actual Weight,Age,Gender,Duration,Heart Rate,BMI,Weather Conditions,Exercise Intensity
0,1,Exercise 2,286.959851,91.892531,96.301115,45,Male,37,170,29.426275,Rainy,5
1,2,Exercise 7,343.453036,64.165097,61.104668,25,Male,43,142,21.286346,Rainy,5
2,3,Exercise 4,261.223465,70.846224,71.766724,20,Male,20,148,27.899592,Cloudy,4
3,4,Exercise 5,127.183858,79.477008,82.984456,33,Male,39,170,33.729552,Sunny,10
4,5,Exercise 10,416.318374,89.960226,85.643174,29,Female,34,118,23.286113,Cloudy,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3859,3860,Exercise 5,154.113144,98.147291,96.745133,20,Female,22,139,32.447764,Cloudy,7
3860,3861,Exercise 4,486.392768,97.598957,92.700573,21,Female,49,160,26.602475,Rainy,5
3861,3862,Exercise 4,264.307731,94.946612,96.778936,57,Male,56,167,31.435348,Rainy,9
3862,3863,Exercise 9,185.951870,64.743906,68.662889,58,Female,60,128,19.774614,Rainy,1


In [2]:
df.isna().sum()

ID                    0
Exercise              0
Calories Burn         0
Dream Weight          0
Actual Weight         0
Age                   0
Gender                0
Duration              0
Heart Rate            0
BMI                   0
Weather Conditions    0
Exercise Intensity    0
dtype: int64

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Define a dictionary of intensity levels and descriptions
intensity_descriptions = {
    1: 'Very low intensity: Suitable for individuals with minimal physical activity, such as walking at a leisurely pace, stretching or yoga, and gentle swimming. However, please note that your preference is prioritized.',
    2: 'Low intensity: Suitable for beginners or individuals with limited physical activity, such as light jogging or running, cycling at a relaxed pace, and beginner\'s aerobics. However, please remember that your preference takes priority.',
    3: 'Moderate intensity: Suitable for individuals who engage in regular physical activity, including brisk walking, dancing, and water aerobics. However, please keep in mind that your preference is given priority.',
    4: 'Medium intensity: Suitable for individuals with moderate fitness and physical activity, such as power walking, cycling at a moderate pace, and Zumba. However, please remember that your preference is given priority.',
    5: 'Moderate to high intensity: Suitable for individuals with moderate to high fitness levels, including jogging or running at a moderate pace, high-intensity interval training (HIIT), and kickboxing. However, please note that your preference is given priority.',
    6: 'High intensity: Suitable for individuals with a high level of fitness and physical activity, such as running at a fast pace, circuit training, and CrossFit. However, please keep in mind that your preference is given priority.',
    7: 'High intensity: Suitable for individuals with a high level of fitness and physical activity, such as advanced HIIT workouts, competitive sports (e.g., soccer, basketball), and spinning or indoor cycling classes. However, please remember that your preference is given priority.',
    8: 'Very high intensity: Suitable for individuals with very high fitness and physical activity, including sprinting or interval sprints, plyometric exercises, and heavy weightlifting. However, please keep in mind that your preference is given priority.',
    9: 'Very higher intensity: Suitable for individuals with very high fitness and physical activity, such as advanced CrossFit workouts, box jumps, and Olympic weightlifting. However, please note that your preference is given priority.',
    10: 'Extremely high intensity: Suitable for athletes or individuals with exceptional fitness levels, including professional sports training, marathon running, and elite-level strength and conditioning programs. However, please remember that your preference is given priority.'
}

# Load the dataset
df = pd.read_csv('exercise_dataset.csv')

# Drop unnecessary columns
df.drop(['ID'], axis=1, inplace=True)



# Remove outliers
def remove_outliers(df, columns, threshold=3):
    df_cleaned = df.copy()
    for column in columns:
        z_scores = (df[column] - df[column].mean()) / df[column].std()
        outliers = df_cleaned.loc[abs(z_scores) > threshold]
        df_cleaned = df_cleaned.drop(outliers.index)
    return df_cleaned

columns_to_remove_outliers = ['Calories Burn', 'Dream Weight', 'Actual Weight', 'Age', 'Duration', 'Heart Rate', 'BMI', 'Exercise Intensity']
df_cleaned = remove_outliers(df, columns_to_remove_outliers)

# Calculate the 'Height' and 'maximum Heart Rate' columns
import math
df_cleaned['Height'] = df_cleaned.apply(lambda row: math.sqrt(row['Actual Weight'] / row['BMI']), axis=1)
df_cleaned['maximum Heart Rate'] = 220 - df_cleaned['Age']

# Drop unnecessary columns
columns_to_drop = ['Heart Rate', 'BMI', 'Exercise']
df_cleaned.drop(columns=columns_to_drop, inplace=True)

# Split the dataset into features (X) and target variable (y)
X = df_cleaned.drop('Exercise Intensity', axis=1)
y = df_cleaned['Exercise Intensity']

# Encode categorical features
categorical_features = ['Gender', 'Weather Conditions']
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_categorical = ohe.fit_transform(X[categorical_features])
X_categorical_df = pd.DataFrame(X_categorical, columns=ohe.get_feature_names_out(categorical_features))

# Scale numerical features
numerical_features = ['Calories Burn', 'Dream Weight', 'Actual Weight', 'Age', 'Duration', 'maximum Heart Rate', 'Height']
scaler = StandardScaler()
X_numerical = scaler.fit_transform(X[numerical_features])
X_numerical_df = pd.DataFrame(X_numerical, columns=numerical_features)

# Concatenate the encoded categorical features and scaled numerical features
X_encoded_scaled = pd.concat([X_categorical_df, X_numerical_df], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded_scaled, y, test_size=0.2, random_state=1)

# Instantiate the Random Forest Regressor
final_model = RandomForestRegressor(max_depth=5, n_estimators=200, random_state=2)

# Fit the model on the training data
final_model.fit(X_train, y_train)

# Make predictions on the test data
rf_y_pred = final_model.predict(X_test)

# Evaluate the model
final_rmse = np.sqrt(mean_squared_error(y_test, rf_y_pred))
print("RMSE - Random Forest:", final_rmse)

# User Input
dream_weight = float(input("Enter your dream weight: "))
actual_weight = float(input("Enter your actual weight: "))
age = float(input("Enter your age: "))
gender = input("Enter your Gender (M/F): ")
duration = float(input("Enter your training session duration (minutes): "))
height = float(input("Enter your height in meters: "))
max_heart_rate = 220 - age
calories_to_burn = float(input("Enter the calories burned: "))
weather_conditions = input("Enter your suitable training weather condition: ")

# Create a dataframe with user input
user_input = pd.DataFrame(
    [[calories_to_burn, dream_weight, actual_weight, age, duration, height, max_heart_rate, gender, weather_conditions]],
    columns=['Calories Burn', 'Dream Weight', 'Actual Weight', 'Age', 'Duration', 'Height', 'maximum Heart Rate','Gender',  'Weather Conditions'])

# Encode categorical features in user input
user_input_categorical = ohe.transform(user_input[categorical_features])
user_input_categorical_df = pd.DataFrame(user_input_categorical, columns=ohe.get_feature_names_out(categorical_features))

# Scale numerical features in user input
user_input_numerical = scaler.transform(user_input[numerical_features])
user_input_numerical_df = pd.DataFrame(user_input_numerical, columns=numerical_features)

# Concatenate the encoded categorical features and scaled numerical features for user input
user_input_encoded_scaled = pd.concat([user_input_categorical_df, user_input_numerical_df], axis=1)

# Make predictions on the user input
predicted_intensity_selected = int(round(final_model.predict(user_input_encoded_scaled)[0]))

# Retrieve the description based on the predicted intensity level
description = intensity_descriptions.get(predicted_intensity_selected, "Unknown")

# Display the recommendation and intensity description
print("Recommended Exercise Intensity:", predicted_intensity_selected)
print("Intensity Description:", description)
print("Maximum Heart rate:", max_heart_rate)
print("Calories to burn (Kcal):", calories_to_burn)


c:\Users\hp\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


RMSE - Random Forest: 2.8962802805407746
Recommended Exercise Intensity: 5
Intensity Description: Moderate to high intensity: Suitable for individuals with moderate to high fitness levels, including jogging or running at a moderate pace, high-intensity interval training (HIIT), and kickboxing. However, please note that your preference is given priority.
Maximum Heart rate: 187.0
Calories to burn (Kcal): 127.0
